In [1]:
'''
Project: P300: Speller Matrix
Authors:
    -Natascha Niessen
    -Pamela Reyna
    -Oscar Soto
    -Alena Starikova
Description: Offline analysis and visualization of training EEG data for a P300 application
'''

'\nProject: P300: Speller Matrix\nAuthors:\n    -Natascha Niessen\n    -Pamela Reyna\n    -Oscar Soto\n    -Alena Starikova\nDescription: Offline analysis and visualization of training EEG data for a P300 application\n'

In [2]:
import numpy as np
import pyxdf
import mne
import joblib
from mne.datasets import misc
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold, train_test_split
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

matplotlib.use('TkAgg')

Read NPY file

In [3]:
#Load npy files
eeg_samples = np.transpose(np.load('eeg_samples.npy'))
eeg_timestamps = np.load('eeg_timestamps.npy')
marker_samples = np.load('marker_samples.npy')
marker_timestamps = np.load('marker_timestamps.npy')
corrected_marker_timestamps = np.load('corrected_marker_timestamps.npy')

eeg_data = mne.filter.filter_data(eeg_samples.astype(np.float64),500,2,16)
print(np.shape(eeg_timestamps))

Setting up band-pass filter from 2 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 825 samples (1.650 s)



(19501,)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


We need to create an mne object in order to use filter and epoch functions. Extract the elements from the dictiornaries.
streams[0] contains info about markers and streams[1] contains info about EEG data

**Creates RAW data object**

In [4]:
#When reading an XDF file you have streams. In our case, each stream corresponds to an outlet recorded in lab recorder. 
#However, the order in which those structures are saved in the xdf file is not always the same. To identify it, it is 
#necessary to consult the 'type' key as shown below
xdf_file_path = "data/sub-P001_ses-S200_task-Default_run-001_eeg.xdf"
streams, header = pyxdf.load_xdf(xdf_file_path)
for stream in range(len(streams)): #This for loop will iterate over all the streams we have. Normally we only have 2, the EEG signal and the markers. However, we parameterize it so that the code is reusable.
    type = streams[stream]['info']['type'][0]
    if type == 'EEG':
        eeg_stream_idx = stream
    if type == 'Markers':
        marker_stream_idx = stream
        
print("Number of streams found: " + str(len(streams)))

eeg_data_raw = streams[eeg_stream_idx]['time_series']
sampling_rate = streams[eeg_stream_idx]['info']['nominal_srate'][0]

#Extract channel names
number_channels = np.shape(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'])[0]
ch_names = []
ch_pos = {}
for i in range(number_channels):
    channel_label = streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]
    pos_x = np.float64(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['location'][0]['X'][0])#Get X channel position
    pos_y = np.float64(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['location'][0]['Y'][0])#Get Y channel position
    pos_z = np.float64(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['location'][0]['Z'][0])#Get Z channel position
    
    ch_names.append(channel_label)
    ch_pos[channel_label] = [pos_x,pos_y,pos_z]
   
ch_types = ['eeg'] * len(ch_names)


Stream 2: Calculated effective sampling rate 8.8342 Hz is different from specified rate 500.0000 Hz.


Number of streams found: 2


**Create events to use for the epoching phase**

In [5]:
#https://mne.tools/stable/generated/mne.Epochs.html
marker_id = np.squeeze(marker_samples)
marker_timestamps_scale =  marker_timestamps - eeg_timestamps[0]

#Creates event array, for this we need event_sample and event_id. Our event sample will be int(marker_timestamps[i]) and event_id as define below
events = []
for i,marker in enumerate(marker_id):
    event_id = {'Init': 10, 'StartTask': 11, 'End': 12, 'S0': 0, 'S1': 1, 'S2': 2}[marker] #Iterates over marker_id and every time it finds an S10 it assigns a 1 to event_id, 
                                                  #every time it finds an S11 it assigns a 2 to event_id list and then append the value to
                                                  #events
    #events.append([int((marker_timestamps[i]-6.889)*1000), 0, event_id])
    events.append([marker_timestamps_scale[i], 0, event_id])
                                          
#Why do we use numeric values ​​instead of directly using S10 or S11? 
# Some functions can expect number values ​​as parameters, so it is more convenient to use a dictionary and assign 
# S10 and S11 a numeric representation. In case we need to use a string, it is easier to pass this number value 
# to a string or use the dictionary to index its key (S10 or S11)

In [6]:
duplicate_events = []
timestamps = [event[0] for event in events]

for i in range(len(events)):
    if timestamps.count(events[i][0]) > 1:
        duplicate_events.append(i)

print("Events with the same timestamps:")
for index in duplicate_events:
    print(events[index])

Events with the same timestamps:


In [7]:
print(np.shape(eeg_data))

(24, 19501)


In [8]:
# Create info object
info = mne.create_info(ch_names, sfreq=500, ch_types=ch_types)
#Set channel positions
montage = mne.channels.make_dig_montage(ch_pos)
info.set_montage(montage)

raw = mne.io.RawArray(eeg_data, info)

Creating RawArray with float64 data, n_channels=24, n_times=19501
    Range : 0 ... 19500 =      0.000 ...    39.000 secs
Ready.


C:\Users\ohskr\AppData\Local\Temp\ipykernel_30344\3522202612.py:5: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  info.set_montage(montage)


Plot EEG signal

In [9]:
raw.plot(events=np.asarray(events),scalings=dict(eeg=1e1),show_options=True)

Using matplotlib as 2D backend.


<MNEBrowseFigure size 1707x897 with 4 Axes>

Channels marked as bad:
none


Plot PSD

In [10]:
raw.compute_psd(fmax = 30).plot()

Effective window size : 4.096 (s)


C:\Users\ohskr\AppData\Local\Temp\ipykernel_30344\547714129.py:1: RuntimeWarning: in version 1.5, the default behavior of Spectrum.plot() will change so that bad channels will be shown by default. To keep the old default behavior (and silence this warning), explicitly pass `picks='data', exclude='bads'`.
  raw.compute_psd(fmax = 30).plot()


<MNELineFigure size 1000x350 with 2 Axes>

**Implement and call functions for epoching**

In [21]:
def find_nearest_indices(arr1, arr2):
    indices = []
    for value in arr1:
        abs_diff = np.abs(arr2 - value)
        nearest_index = np.argmin(abs_diff) #The minimum absolute value between the arrays is calculated. For example if arr2 has elements 454 and 456 and value is 455, 
                                            #the subtraction abs(454-455) = 1 and abs(456-455) = 1, how then select only one of them? np.argmin() returns only the index of 
                                            #the first minimum found in case there are several minimum values ​​in the array.
        indices.append(nearest_index)
    return indices

def epoch(raw,marker_timestamps,marker_id,marker,tmin,tmax,sampling_rate):
    #Let's go and find how many samples tmin and tmax are equal to. Remember: Samples are integers/discrete values, thats why we have to cast to int
    tmin_samples = np.abs(int(tmin*int(sampling_rate)))
    tmax_samples = np.abs(int(tmax*int(sampling_rate)))
    marker_index = np.where(marker_id == marker)
    time_points = np.round(marker_timestamps[np.where(marker_id == marker)],3)
    indices = find_nearest_indices(time_points,raw.times)
    epochs = []
    eeg_data = np.array(raw.get_data())
    for trial in range(len(indices)):
        epochs.append(eeg_data[:,indices[trial] - tmin_samples:indices[trial] + tmax_samples])
    
    epochs = np.array(epochs)#Convert to numpy array
    return epochs, marker_index

tmin = -0.5
tmax = 1
epochsS0,S0_idx = epoch(raw,marker_timestamps_scale,marker_id,'S0',tmin,tmax,500)
epochsS1,S1_idx = epoch(raw,marker_timestamps_scale,marker_id,'S1',tmin,tmax,500)
epochsS2,S2_idx = epoch(raw,marker_timestamps_scale,marker_id,'S2',tmin,tmax,500)

**Visualization**

In [12]:
time = np.arange(tmin,tmax,1/int(500)) #This is not the general time but an snapshop from -0.500 to 1.000 using the sample sampling frequency used in data collection
S0_avg = np.mean(epochsS0,axis = 0)
S1_avg = np.mean(epochsS1,axis = 0)
S2_avg = np.mean(epochsS2,axis = 0)

#Select a window to look for maximum
lower_time_window = 0.250
upper_time_window = 0.305
lower_idx = np.where(time >= lower_time_window)[0][0]
upper_idx = np.where(time <= upper_time_window)[0][-1]

#Set plot setting (title, number of subplots, etc)
fig, axs = plt.subplots(int(len(ch_names)/4), int(len(ch_names)/6))
fig.suptitle("Epoch avg for event and non event trials")
fig.subplots_adjust(hspace=0.9)
plot_next_row = 0
plot_next_column = 0
#Create plots
for ch in range(int(len(ch_names))):
    if ch%6 == 0 and ch != 0: #Each 6 rows update the indexes for subplots
        plot_next_row = 0
        plot_next_column = plot_next_column + 1 
    axs[plot_next_row, plot_next_column].plot(time,S0_avg[ch,:], color = 'lightgreen',label="S0")
    axs[plot_next_row, plot_next_column].plot(time,S1_avg[ch,:], color = 'blue',label="S1")
    axs[plot_next_row, plot_next_column].plot(time,S2_avg[ch,:], color = 'red',label="S2")
    
    #Create one legend for all plots
    if (plot_next_row == 0 and plot_next_column == 0):
        legend_labels = ['S0', 'S1', 'S2']
        axs[plot_next_row, plot_next_column].legend(labels=legend_labels, loc='upper center', bbox_to_anchor=(-0.4, 1), ncol=1)
    
    #Add titles, time value and set grid
    axs[plot_next_row, plot_next_column].set_title('Ch ' + str(ch + 1) + ': ' + ch_names[ch])#Title
    axs[plot_next_row, plot_next_column].grid(True)#grid on
    
    plot_next_row = plot_next_row + 1



plt.show()

**Online method**

In [24]:
# Comparar si los arreglos son iguales
are_equal = np.array_equal(eeg_timestamps_marker_S0, S0_idx)

# Imprimir el resultado
if are_equal:
    print("Los arreglos son iguales.")
else:
    print("Los arreglos son diferentes.")

Los arreglos son diferentes.


In [25]:
print(eeg_timestamps_marker_S0)
print(S0_idx)

[15500 16004 16006 16500 16504 16506 17002 17005 17007 17500 17505 17506
 18000 18005 18500]
[16031, 16318, 16433, 16605, 16835, 16949, 17179, 17351, 17465, 17580, 17867, 17924, 18096, 18385, 18558]


In [20]:
low_bandpass_fr = 2
high_bandpass_fr = 16
eeg_samples_filter = mne.filter.filter_data(eeg_samples.astype(np.float64),sampling_rate,low_bandpass_fr,high_bandpass_fr)
#To do the epoching we will first split the marker_sample array according to its 
# marker, S0, S1 or S2. By finding the indices of each one, we can use them to 
# index their corresponding timestamps. That is, obtain the times in which each 
# marker occurred. Having this information, we only have to search for these times 
# within the eeg_samples timestamp, obtain their indices and then use them 
# to finally index the eeg_samples 
markers_S0_idx = np.where(marker_samples == "S0")[0]
markers_S1_idx = np.where(marker_samples == "S1")[0]
markers_S2_idx = np.where(marker_samples == "S2")[0]

timestamps_S0 = corrected_marker_timestamps[markers_S0_idx]
timestamps_S1 = corrected_marker_timestamps[markers_S1_idx]
timestamps_S2 = corrected_marker_timestamps[markers_S2_idx]

eeg_timestamps_marker_S0 = np.where(np.isin(eeg_timestamps, timestamps_S0))[0]
eeg_timestamps_marker_S1 = np.where(np.isin(eeg_timestamps, timestamps_S1))[0]
eeg_timestamps_marker_S2 = np.where(np.isin(eeg_timestamps, timestamps_S2))[0]

game_trials = len(eeg_timestamps_marker_S0)

tmin = 0
tmax = 1
tmin_samples = np.abs(int(tmin*int(sampling_rate)))
tmax_samples = np.abs(int(tmax*int(sampling_rate)))

epochs_S0 = []
epochs_S1 = []
epochs_S2 = []
for game_trial in range(game_trials):
    epochs_S0.append(eeg_samples_filter[:,eeg_timestamps_marker_S0[game_trial]:eeg_timestamps_marker_S0[game_trial] + tmax_samples])
    epochs_S1.append(eeg_samples_filter[:,eeg_timestamps_marker_S1[game_trial]:eeg_timestamps_marker_S1[game_trial] + tmax_samples])
    epochs_S2.append(eeg_samples_filter[:,eeg_timestamps_marker_S2[game_trial]:eeg_timestamps_marker_S2[game_trial] + tmax_samples])

epochs_S0 = np.array(epochs_S0)
epochs_S1 = np.array(epochs_S1)
epochs_S2 = np.array(epochs_S2)

avg_marker_S0 = np.mean(epochs_S0, axis = 0)
avg_marker_S1 = np.mean(epochs_S1, axis = 0)
avg_marker_S2 = np.mean(epochs_S2, axis = 0)

#Select a window to look for maximum and get the indexes
time = np.arange(tmin,tmax,1/int(sampling_rate))
lower_time_window = 0.250
upper_time_window = 0.305
lower_idx = np.where(time >= lower_time_window)[0][0]
upper_idx = np.where(time <= upper_time_window)[0][-1]

plt.plot(time,avg_marker_S0[21,:],color='lightgreen')
plt.plot(time,avg_marker_S1[21,:],color='blue')
plt.plot(time,avg_marker_S2[21,:],color='red')
legend_labels = ['0', '1', '2']
plt.legend(legend_labels)
plt.show()

Setting up band-pass filter from 2 - 16 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 4.00 Hz (-6 dB cutoff frequency: 18.00 Hz)
- Filter length: 825 samples (1.650 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


**Feature extraction**

In [ ]:
trials_per_task_for_event = 15
trials_per_task_for_non_event = 30
number_of_tasks = 30
number_of_trials_event = number_of_tasks*trials_per_task_for_event
number_of_trials_non_event = number_of_tasks*trials_per_task_for_non_event

_,number_of_ch,number_of_samples = np.shape(event_epochs)

feature_method = "Method 1"

feature_methods = {
    "Method 1": {
        "number": 1,
        "description": "The average of each trial for each task is calculated, and then the maximum value is extracted from each average within the range " + str(lower_time_window)+"s:"+str(upper_time_window) +"s."
    },
    "Method 2": {
        "number": 2,
        "description": "The maximum value for each trial is extracted within the range " + str(lower_time_window)+"s:"+str(upper_time_window) +"s."
    }
}

try:
    feature_method_selection = feature_methods[feature_method]["number"]
    if(feature_method_selection == 1):
        description = feature_methods[feature_method]["description"]
        print("Description:", description)
        #Average trial per task (event markers)
        task_counter = 0
        event_avg_per_trial_per_task = np.zeros((number_of_tasks, number_of_ch, number_of_samples))
        feature_1_event = np.zeros((number_of_tasks,number_of_ch))
        for trial_group_event in range(0,number_of_trials_event - 1,trials_per_task_for_event):
            event_avg_per_trial_per_task[task_counter,:,:] = np.mean(event_epochs[trial_group_event:(trial_group_event+trials_per_task_for_event-1),:,:],axis = 0)#Average event trials per task
            feature_1_event[task_counter,:] = np.max(event_avg_per_trial_per_task[task_counter, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each average event trial per task in the given range and store it in the feature vector
            task_counter = task_counter + 1

        feature_1_event = np.array(feature_1_event)

        #Average trial per task (non event markers)
        task_counter = 0
        non_event_avg_per_trial_per_task = np.zeros((number_of_tasks, number_of_ch, number_of_samples))
        feature_1_non_event = np.zeros((number_of_tasks,number_of_ch))
        for trial_group_non_event in range(0,number_of_trials_non_event - 1,trials_per_task_for_non_event):
            non_event_avg_per_trial_per_task[task_counter,:,:] = np.mean(non_event_epochs[trial_group_non_event:(trial_group_non_event+trials_per_task_for_non_event-1),:,:],axis = 0)#Average event trials per task
            feature_1_non_event[task_counter,:] = np.max(non_event_avg_per_trial_per_task[task_counter, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each average non event trial per task in the given range and store it in the feature vector
            task_counter = task_counter + 1

        feature_1_non_event = np.array(feature_1_non_event)

        #Create labels for event and non event class
        labels_event = np.full((len(feature_1_event), 1), 1).astype(int)
        labels_non_event = np.full((len(feature_1_non_event), 1), -1).astype(int)

        #Concatenate vectors in a single training data matrix
        features = np.concatenate((feature_1_event, feature_1_non_event), axis=0)
        labels = np.concatenate((labels_event, labels_non_event), axis=0)
        training_data = np.concatenate((features, labels), axis=1)
    elif (feature_method_selection == 2):
        description = feature_methods[feature_method]["description"]
        print("Description:", description)
        feature_1_event = np.zeros((number_of_trials_event,number_of_ch))
        for trial in range(number_of_trials_event):
            feature_1_event[trial,:] = np.max(event_epochs[trial, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each trial in the given range and store it in the feature vector

        feature_1_event = np.array(feature_1_event)

        feature_1_non_event = np.zeros((number_of_trials_non_event,number_of_ch))
        for trial in range(number_of_trials_non_event):
            feature_1_non_event[trial,:] = np.max(non_event_epochs[trial, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each trial in the given range and store it in the feature vector

        feature_1_non_event = np.array(feature_1_non_event)

        #Create labels for event and non event class
        labels_event = np.full((len(feature_1_event), 1), 1).astype(int)
        labels_non_event = np.full((len(feature_1_non_event), 1), -1).astype(int)

        #Concatenate vectors in a single training data matrix
        features = np.concatenate((feature_1_event, feature_1_non_event), axis=0)
        labels = np.concatenate((labels_event, labels_non_event), axis=0)
        training_data = np.concatenate((features, labels), axis=1)
except:
    print("That feature extraction method is not valid")
    for method, details in feature_methods.items():
        print(method, "Description:", details["description"])

**Feature visualization**

In [ ]:
if (feature_method == "Method 1"):
    time = np.arange(tmin,tmax,1/int(sampling_rate))

    for ch in range(len(ch_names)):
        
        #Set plot setting (title, number of subplots, etc)
        fig, axs = plt.subplots(int(number_of_tasks/3), int(number_of_tasks/10))
        fig.suptitle("Avg trial per task channel " + str(ch + 1) + ': ' + ch_names[ch])
        fig.subplots_adjust(hspace=0.5)
        plot_next_row = 0
        plot_next_column = 0
        
        #Create plots
        for task in range(number_of_tasks):
            max_idx = np.where(event_avg_per_trial_per_task[task, ch, :] == feature_1_event[task,ch])[0][0]
            if task%3 == 0 and task != 0: #Each 6 rows update the indexes for subplots
                plot_next_column = 0
                plot_next_row = plot_next_row + 1
            axs[plot_next_row, plot_next_column].plot(time,event_avg_per_trial_per_task[task,ch,:], color = 'lightgreen',label="Event")
            axs[plot_next_row, plot_next_column].plot(time,non_event_avg_per_trial_per_task[task,ch,:], color = 'blue',label="Non Event")
            
            #Create one legend for all plots
            if (plot_next_row == 0 and plot_next_column == 0):
                legend_labels = ['Event', 'Non Event']
                axs[plot_next_row, plot_next_column].legend(labels=legend_labels, loc='upper center', bbox_to_anchor=(-0.35, 1.2), ncol=1)
            
            #Add markers at maximum time points in the given range (lower_idx:upper_idx)
            axs[plot_next_row, plot_next_column].plot(time[max_idx], event_avg_per_trial_per_task[task,ch,max_idx], marker='x', color='red')
            axs[plot_next_row, plot_next_column].plot(time[max_idx], non_event_avg_per_trial_per_task[task,ch,max_idx], marker='x', color='red')
            
            #Add titles, time value and set grid
            axs[plot_next_row, plot_next_column].set_title('Task ' + str(task + 1), fontsize=8)#Title
            axs[plot_next_row, plot_next_column].text(time[max_idx], 0, str(np.round(time[max_idx],3)), fontsize=9, color='black', ha='center', va='bottom')#Time value
            axs[plot_next_row, plot_next_column].grid(True)#grid on
            
            plot_next_column = plot_next_column + 1
            
    plt.show()
else:
    print("Visualization is not implemented for this method")

**Feature evaluation**

In [ ]:
def fisherrank(featv, labels):
    nfeat = featv.shape[1]
    c1 = featv[labels == -1, :]
    c2 = featv[labels == 1, :]
    
    d = np.zeros(nfeat)
    
    for f in range(nfeat):
        d[f] = ((np.mean(c1[:, f]) - np.mean(c2[:, f]))**2) / (np.var(c1[:, f]) + np.var(c2[:, f]))
    
    rank = np.argsort(d)[::-1]
    
    return d, rank

d, rank = fisherrank(training_data[:,:-1], training_data[:,-1])
#d, rank = fisherrank(normalize(training_data[:,:-1],axis = 1), training_data[:,-1])
criterion_desc = np.sort(d)[::-1]

# Plot criterion_desc
plt.subplot(1, 2, 1)
plt.plot(criterion_desc)
plt.grid(True)
plt.xlabel('Ranked Features')
plt.ylabel('Fisher Score')

# Scatter plot
event_length = len(feature_1_event)
non_event_length = len(feature_1_non_event)

plt.subplot(1, 2, 2)
plt.scatter(training_data[:event_length - 1, rank[0]], training_data[:event_length - 1, rank[1]], color='blue')
plt.scatter(training_data[event_length:event_length+non_event_length - 1, rank[0]], training_data[event_length:event_length+non_event_length - 1, rank[1]], color='red')
plt.xlabel("1. Best Feature")
plt.ylabel("2. Best Feature")
plt.grid(True)

plt.show()

**Training**

In [ ]:
trials, _ = np.shape(training_data)

# Select the best features in such a way that there are at least 10 times more observations than features
number_of_features_to_pick = 0.1*trials
features_to_pick_idx = np.arange(number_of_features_to_pick).astype(int)

#Save best features index in txt file so we can use it for online data
file_path = '../online_pr/trained_model/best_features_idx.txt'
np.savetxt(file_path, rank[features_to_pick_idx], fmt='%d')

#Create a new feature vector with best features for training
best_features = np.concatenate((training_data[:,rank[features_to_pick_idx]], np.reshape(training_data[:,-1],(-1,1))), axis=1)

# Cross validation
k_folds = 10

# Set ratio for test set
test_ratio = 0.1

# We are going to do several repetitions of the cross-validation to have a
# better estimate of the performance
repetitions = 100

MCR_LDA_validation_acc = 0
MCR_SVM_validation_acc = 0
MCR_LDA_test_acc = 0
MCR_SVM_test_acc = 0

# In the following algorithm we will perform an evaluation of the performance of the classifier.
# In order to have more accurate results we will use the cross-validation technique with kfolds = 10.
# In addition, we will repeat the entire algorithm 100 times to calculate the average performance
# for both the validation set and the test set.
for repetition in range(repetitions):
    # Get a randomly permuted array of indices
    shuffled_indices = np.random.permutation(best_features.shape[0])

    # Shuffle the rows of the array using the shuffled indices
    random_features = best_features[shuffled_indices, :]

    #Normalization
    norm_features = random_features[:,:-1]
    #norm_features = normalize(random_features[:,:-1],axis=0)
    
    # Generate random indices from 0 to rows for test set
    random_test_idx = np.random.choice(trials-1, size=int(trials*test_ratio), replace=False)
    
    # Get the indices for training set (this will later be divided into training and validation set)
    random_training_idx = np.setdiff1d(np.arange(trials), random_test_idx)

    features_training_set_aux = norm_features[random_training_idx]
    labels_training_set_aux = random_features[random_training_idx,-1]#norm_features just contains features and no labels thats why we use random_features here, that has the same order than norm_features
    features_test_set = norm_features[random_test_idx]
    labels_test_set = random_features[random_test_idx,-1]#norm_features just contains features and no labels thats why we use random_features here, that has the same order than norm_features

    MCR_accumulator_LDA = 0
    MCR_accumulator_SVM = 0

    # Cross-validation
    cvIndices = KFold(n_splits=k_folds).split(features_training_set_aux)
    for i in range(k_folds):

        train_indices, validation_indices = next(cvIndices)

        features_training_set = features_training_set_aux[train_indices]
        labels_training_set = labels_training_set_aux[train_indices]
        features_validation_set = features_training_set_aux[validation_indices]
        labels_validation_set = labels_training_set_aux[validation_indices]

        # Train and validate an LDA model for this kfold
        lda_model = LinearDiscriminantAnalysis()
        lda_model.fit(features_training_set, labels_training_set)
        validation_LDA = lda_model.predict(features_validation_set)

        # Train and validate a SVM model for this kfold
        svm_model = SVC()
        svm_model.fit(features_training_set, labels_training_set)
        validation_SVM = svm_model.predict(features_validation_set)

        # Misclassification rate (MCR)
        misclassification_LDA = np.sum(validation_LDA != labels_validation_set)
        misclassification_SVM = np.sum(validation_SVM != labels_validation_set)

        # Store the validation performance for this kfold
        MCR_LDA = misclassification_LDA / len(labels_validation_set)
        MCR_SVM = misclassification_SVM / len(labels_validation_set)

        MCR_accumulator_LDA = MCR_accumulator_LDA + MCR_LDA
        MCR_accumulator_SVM = MCR_accumulator_SVM + MCR_SVM

    # Store the validation performance for this repetition
    MCR_Avg_LDA = MCR_accumulator_LDA / k_folds
    MCR_LDA_validation_acc = MCR_LDA_validation_acc + MCR_Avg_LDA

    MCR_Avg_SVM = MCR_accumulator_SVM / k_folds
    MCR_SVM_validation_acc = MCR_SVM_validation_acc + MCR_Avg_SVM

    # Test the models
    test_LDA = lda_model.predict(features_test_set)
    test_SVM = svm_model.predict(features_test_set)

    # Store the test performance for this repetition
    MCR_LDA_test_repetition = np.sum(test_LDA != labels_test_set) / len(labels_test_set)
    MCR_LDA_test_acc = MCR_LDA_test_acc + MCR_LDA_test_repetition

    MCR_SVM_test_repetition = np.sum(test_SVM != labels_test_set) / len(labels_test_set)
    MCR_SVM_test_acc = MCR_SVM_test_acc + MCR_SVM_test_repetition

# Show average performance for validation and test sets
MCR_LDA_validation = MCR_LDA_validation_acc / repetitions
print(f'LDA Model: Average misclassification rate (MCR) for validation set is {MCR_LDA_validation}')

MCR_SVM_validation = MCR_SVM_validation_acc / repetitions
print(f'SVM Model: Average misclassification rate (MCR) for validation set is {MCR_SVM_validation}')

MCR_LDA_test = MCR_LDA_test_acc / repetitions
print(f'LDA Model: Misclassification rate (MCR) for test set is {MCR_LDA_test}')

MCR_SVM_test = MCR_SVM_test_acc / repetitions
print(f'SVM Model: Misclassification rate (MCR) for test set is {MCR_SVM_test}')


**Choose model and train classifier with full training set**

In [ ]:
# Get a randomly permuted array of indices
shuffled_indices = np.random.permutation(best_features.shape[0])

# Shuffle the rows of the array using the shuffled indices
random_features = best_features[shuffled_indices, :]

#Normalization
norm_features = random_features[:,:-1]
#norm_features = normalize(random_features[:,:-1],axis=0)

#Train model with full training data set
#svm_model = SVC()
#svm_model.fit(norm_features, random_features[:,-1])

lda_model = LinearDiscriminantAnalysis()
lda_model.fit(norm_features, random_features[:,-1])

#Save model for future use in online sessions
file_path = '../online_pr/trained_model/lda_model.pkl'
joblib.dump(lda_model, file_path)


**Test classifier with unseen data**

In [ ]:
#Load and process file
#xdf_file_path = "data/sub-P001_ses-S101_task-Default_run-001_eeg.xdf"
#xdf_file_path = "data/sub-P001_ses-S200_task-Default_run-001_eeg.xdf"
#xdf_file_path = "data/sub-P001_ses-S201_FaceDelay_task-Default_run-001_eeg.xdf"
xdf_file_path = "data/sub-P001_ses-S202_NoBlackSBetweenTrials_task-Default_run-001_eeg.xdf"
#xdf_file_path = "data/sub-P001_ses-S204_Just_Highlight_Interest_Number_task-Default_run-001_eeg.xdf"
streams, header = pyxdf.load_xdf(xdf_file_path)

#When reading an XDF file you have streams. In our case, each stream corresponds to an outlet recorded in lab recorder. 
#However, the order in which those structures are saved in the xdf file is not always the same. To identify it, it is 
#necessary to consult the 'type' key as shown below

for stream in range(len(streams)): #This for loop will iterate over all the streams we have. Normally we only have 2, the EEG signal and the markers. However, we parameterize it so that the code is reusable.
    type = streams[stream]['info']['type'][0]
    if type == 'EEG':
        eeg_stream_idx = stream
    if type == 'Markers':
        marker_stream_idx = stream
        
print("Number of streams found: " + str(len(streams)))

#Creates RAW data object
eeg_data_raw = streams[eeg_stream_idx]['time_series']
sampling_rate = streams[eeg_stream_idx]['info']['nominal_srate'][0]
eeg_data = mne.filter.filter_data(eeg_data_raw.T.astype(np.float64),sampling_rate,2,16)
eeg_timestamps = streams[eeg_stream_idx]['time_stamps']

#Extract channel names
number_channels = np.shape(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'])[0]
ch_names = []
ch_pos = {}
for i in range(number_channels):
    channel_label = streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0]
    pos_x = np.float64(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['location'][0]['X'][0])#Get X channel position
    pos_y = np.float64(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['location'][0]['Y'][0])#Get Y channel position
    pos_z = np.float64(streams[eeg_stream_idx]['info']['desc'][0]['channels'][0]['channel'][i]['location'][0]['Z'][0])#Get Z channel position
    
    ch_names.append(channel_label)
    ch_pos[channel_label] = [pos_x,pos_y,pos_z]
   
ch_types = ['eeg'] * len(ch_names)

#Create events to use for the epoching phase
#https://mne.tools/stable/generated/mne.Epochs.html
marker_id = np.squeeze(streams[marker_stream_idx]['time_series'])
marker_timestamps = streams[marker_stream_idx]['time_stamps']
marker_timestamps =  marker_timestamps - eeg_timestamps[0]

#Creates event array, for this we need event_sample and event_id. Our event sample will be int(marker_timestamps[i]) and event_id as define below
events = []
for i,marker in enumerate(marker_id):
    event_id = {'S10': 1, 'S11': 2}[marker] #Iterates over marker_id and every time it finds an S10 it assigns a 1 to event_id, 
                                                  #every time it finds an S11 it assigns a 2 to event_id list and then append the value to
                                                  #events
    #events.append([int((marker_timestamps[i]-6.889)*1000), 0, event_id])
    events.append([marker_timestamps[i], 0, event_id])
                                          
#Why do we use numeric values ​​instead of directly using S10 or S11? 
# Some functions can expect number values ​​as parameters, so it is more convenient to use a dictionary and assign 
# S10 and S11 a numeric representation. In case we need to use a string, it is easier to pass this number value 
# to a string or use the dictionary to index its key (S10 or S11)

# Create info object
info = mne.create_info(ch_names, sfreq=sampling_rate, ch_types=ch_types)
#Set channel positions
montage = mne.channels.make_dig_montage(ch_pos)
info.set_montage(montage)

raw = mne.io.RawArray(eeg_data, info)

#Epoching
event_epochs = epoch(raw,marker_timestamps,marker_id,'S10',tmin,tmax,sampling_rate)
non_event_epochs = epoch(raw,marker_timestamps,marker_id,'S11',tmin,tmax,sampling_rate)

#Feature extraction

try:
    feature_method_selection = feature_methods[feature_method]["number"]
    if(feature_method_selection == 1):
        description = feature_methods[feature_method]["description"]
        print("Description:", description)
        #Average trial per task (event markers)
        task_counter = 0
        event_avg_per_trial_per_task = np.zeros((number_of_tasks, number_of_ch, number_of_samples))
        feature_1_event = np.zeros((number_of_tasks,number_of_ch))
        for trial_group_event in range(0,number_of_trials_event - 1,trials_per_task_for_event):
            event_avg_per_trial_per_task[task_counter,:,:] = np.mean(event_epochs[trial_group_event:(trial_group_event+trials_per_task_for_event-1),:,:],axis = 0)#Average event trials per task
            feature_1_event[task_counter,:] = np.max(event_avg_per_trial_per_task[task_counter, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each average event trial per task in the given range and store it in the feature vector
            task_counter = task_counter + 1

        feature_1_event = np.array(feature_1_event)

        #Average trial per task (non event markers)
        task_counter = 0
        non_event_avg_per_trial_per_task = np.zeros((number_of_tasks, number_of_ch, number_of_samples))
        feature_1_non_event = np.zeros((number_of_tasks,number_of_ch))
        for trial_group_non_event in range(0,number_of_trials_non_event - 1,trials_per_task_for_non_event):
            non_event_avg_per_trial_per_task[task_counter,:,:] = np.mean(non_event_epochs[trial_group_non_event:(trial_group_non_event+trials_per_task_for_non_event-1),:,:],axis = 0)#Average event trials per task
            feature_1_non_event[task_counter,:] = np.max(non_event_avg_per_trial_per_task[task_counter, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each average non event trial per task in the given range and store it in the feature vector
            task_counter = task_counter + 1

        feature_1_non_event = np.array(feature_1_non_event)

        #Create labels for event and non event class
        labels_event = np.full((len(feature_1_event), 1), 1).astype(int)
        labels_non_event = np.full((len(feature_1_non_event), 1), -1).astype(int)

        #Concatenate vectors in a single training data matrix
        features = np.concatenate((feature_1_event, feature_1_non_event), axis=0)
        labels = np.concatenate((labels_event, labels_non_event), axis=0)
        training_data = np.concatenate((features, labels), axis=1)
    elif (feature_method_selection == 2):
        description = feature_methods[feature_method]["description"]
        print("Description:", description)
        feature_1_event = np.zeros((number_of_trials_event,number_of_ch))
        for trial in range(number_of_trials_event):
            feature_1_event[trial,:] = np.max(event_epochs[trial, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each trial in the given range and store it in the feature vector

        feature_1_event = np.array(feature_1_event)

        feature_1_non_event = np.zeros((number_of_trials_non_event,number_of_ch))
        for trial in range(number_of_trials_non_event):
            feature_1_non_event[trial,:] = np.max(non_event_epochs[trial, :, lower_idx:upper_idx], axis=1)#Get the maximum amplitude for each trial in the given range and store it in the feature vector

        feature_1_non_event = np.array(feature_1_non_event)

        #Create labels for event and non event class
        labels_event = np.full((len(feature_1_event), 1), 1).astype(int)
        labels_non_event = np.full((len(feature_1_non_event), 1), -1).astype(int)

        #Concatenate vectors in a single training data matrix
        features = np.concatenate((feature_1_event, feature_1_non_event), axis=0)
        labels = np.concatenate((labels_event, labels_non_event), axis=0)
        training_data = np.concatenate((features, labels), axis=1)
except:
    print("That feature extraction method is not valid")
    for method, details in feature_methods.items():
        print(method, "Description:", details["description"])

#Get best features base on fisher rank during training phase
best_features = np.concatenate((training_data[:,rank[features_to_pick_idx]], np.reshape(training_data[:,-1],(-1,1))), axis=1)

# Get a randomly permuted array of indices
shuffled_indices = np.random.permutation(best_features.shape[0])

# Shuffle the rows of the array using the shuffled indices
random_features = best_features[shuffled_indices, :]

#Normalization
norm_features = random_features[:,:-1]
#norm_features = normalize(random_features[:,:-1],axis=0)

#Predict labels and compare with true labels
#test_SVM = svm_model.predict(norm_features)
#MCR_SVM_test_unseen = np.sum(test_SVM != random_features[:,-1]) / len(random_features[:,-1])
#print(f'SVM Model: Misclassification rate (MCR) for unseen set is {MCR_SVM_test_unseen}')

test_LDA = lda_model.predict(norm_features)
MCR_LDA_test_unseen = np.sum(test_LDA != random_features[:,-1]) / len(random_features[:,-1])
print(f'LDA Model: Misclassification rate (MCR) for unseen set is {MCR_LDA_test_unseen}')